In [1]:
#Implement a perceptron from scratch 
import numpy as np
import os
import cv2

# Load the training data from the folder
train_folder = '/home/vaishnavi-moutam/Downloads/final/Training1'
X_train = []
y_train = []
for file in os.listdir(train_folder):
    img = cv2.imread(os.path.join(train_folder, file))
    if img is not None:  # Check if image is not empty
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_train.append(img)  # Append the flattened image to the list
        y_train.append(1)  # or 0, depending on the class label
    else:
        print(f"Error reading file {file}")  # Print an error message if the image is empty

# Convert the lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Initialize the weights and bias
weights = np.random.rand(X_train.shape[1])
bias = 0

# Set the learning rate and number of iterations
learning_rate = 0.01
n_iters = 1

# Train the model
for iter in range(n_iters):
    for i, x_i in enumerate(X_train):
        # Calculate the predicted output
        predicted = np.where(np.dot(x_i, weights) + bias >= 0, 1, -1)
        # Update the weights and bias
        update = learning_rate * (y_train[i] - predicted)
        weights += update * x_i
        bias += update
    # Calculate the accuracy and loss
    accuracy = np.mean(predicted == y_train)
    loss = np.mean((predicted - y_train) ** 2)
    # Print the accuracy and loss at each iteration
    print(f'Iteration {iter+1}, Accuracy: {accuracy:.2f}, Loss: {loss:.2f}')

# Save the trained model
np.save('weights.npy', weights)
np.save('bias.npy', bias)

# Load the validation data from the folder
val_folder = '/home/vaishnavi-moutam/Downloads/final/Validation1'
X_val = []
y_val = []
for file in os.listdir(val_folder):
    img = cv2.imread(os.path.join(val_folder, file))
    if img is not None:  # Check if image is not empty
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_val.append(img)  # Append the flattened image to the list
        y_val.append(1)  # or 0, depending on the class label
    else:
        print(f"Error reading file {file}")  # Print an error message if the image is empty

# Convert the lists to numpy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)

# Load the trained model
weights = np.load('weights.npy')
bias = np.load('bias.npy')

# Make predictions on the validation set
predictions = np.where(np.dot(X_val, weights) + bias >= 0, 1, -1)
# Calculate the accuracy on the validation set
accuracy = np.mean(predictions == y_val)
print(f'Validation Accuracy: {accuracy:.2f}')

# Load the testing data from the folder
test_folder = '/home/vaishnavi-moutam/Downloads/final/Testing1'
X_test = []
y_test = []
for file in os.listdir(test_folder):
    img = cv2.imread(os.path.join(test_folder, file))
    if img is not None:  # Check if image is not empty
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_test.append(img)  # Append the flattened image to the list
        y_test.append(1)  # or 0, depending on the class label
    else:
        print(f"Error reading file {file}")  # Print an error message if the image is empty

# Convert the lists to numpy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Load the trained model
weights = np.load('weights.npy')
bias = np.load('bias.npy')

# Make predictions on the testing set
predictions = np.where(np.dot(X_test, weights) + bias >= 0, 1, -1)
# Calculate the accuracy on the testing set
accuracy = np.mean(predictions == y_test)
print(f'Testing Accuracy: {accuracy:.2f}')



Iteration 1, Accuracy: 1.00, Loss: 0.00
Validation Accuracy: 1.00
Error reading file evaluation_results_l2.txt
Error reading file final_model_l2.h5
Error reading file evaluation_results_dropout.txt
Error reading file final_model_dropout.h5
Error reading file final_model_l1.h5
Error reading file evaluation_results_l1.txt
Testing Accuracy: 1.00


In [2]:
#Build and train a simple neural network using a framework like TensorFlow or PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from PIL import Image
import os

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

training_path = "/home/vaishnavi-moutam/Downloads/final/Training1"
validation_path = "/home/vaishnavi-moutam/Downloads/final/Validation1"
testing_path = "/home/vaishnavi-moutam/Downloads/final/Testing1"

root = training_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

train_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=True)

root = validation_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

val_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=False)

root = testing_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

test_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=False)

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    # Calculate training accuracy
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    train_acc = correct / total

    # Calculate validation accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = correct / total

    print(f"Epoch [{epoch+1}/10], Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = correct / total
print(f"Testing Accuracy: {test_acc:.4f}")


Skipping /home/vaishnavi-moutam/Downloads/final/Testing1/evaluation_results_dropout.txt: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Testing1/evaluation_results_dropout.txt'
Skipping /home/vaishnavi-moutam/Downloads/final/Testing1/evaluation_results_l1.txt: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Testing1/evaluation_results_l1.txt'
Skipping /home/vaishnavi-moutam/Downloads/final/Testing1/evaluation_results_l2.txt: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Testing1/evaluation_results_l2.txt'
Skipping /home/vaishnavi-moutam/Downloads/final/Testing1/final_model_dropout.h5: cannot find loader for this HDF5 file
Skipping /home/vaishnavi-moutam/Downloads/final/Testing1/final_model_l1.h5: cannot find loader for this HDF5 file
Skipping /home/vaishnavi-moutam/Downloads/final/Testing1/final_model_l2.h5: cannot find loader for this HDF5 file
Epoch [1/10], Loss: 0.4976, Train Acc: 1.0000, Val Acc: 1.0000
Epoch [2/10], Loss:

In [3]:
#Create a multi-layer perceptron (MLP) for digit classification (MNIST dataset)
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define transforms for preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load data from folders
train_data = datasets.MNIST('/home/vaishnavi-moutam/Downloads/final/Training1', download=True, train=True, transform=transform)
validation_data = datasets.MNIST('/home/vaishnavi-moutam/Downloads/final/Validation1', download=True, train=False, transform=transform)
test_data = datasets.MNIST('/home/vaishnavi-moutam/Downloads/final/Testing1', download=True, train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

# Multi-layer perceptron (MLP) model
mlp = nn.Sequential(
    nn.Linear(784, 128),  # input layer (28x28) -> hidden layer (128)
    nn.ReLU(),
    nn.Linear(128, 64),  # hidden layer (128) -> hidden layer (64)
    nn.ReLU(),
    nn.Linear(64, 10)  # hidden layer (64) -> output layer (10)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

# Train the network
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        optimizer.zero_grad()
        outputs = mlp(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/i}')

# Evaluate on training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Training Accuracy: {100 * correct / total} %')

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total} %')

# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation Accuracy: {100 * correct / total} %')

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/train-images-idx3-ubyte.gz to /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/train-labels-idx1-ubyte.gz to /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting /home/vaishnavi-moutam/Downloads/final/Testing1/MNIST/raw/train-labels-idx1-ubyte.gz to /home/vaishnavi-moutam/Downloads/final/Testing1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/vaishnavi-moutam/Downloads/final/Testing1/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/vaishnavi-moutam/Downloads/final/Testing1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/vaishnavi-moutam/Downloads/final/Testing1/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/vaishnavi-moutam/Downloads/final/Testing1/MNIST/raw

Epoch 1, Loss: 0.4304706494432249
Epoch 2, Loss: 0.17740139838212676
Epoch 3, Loss: 0.129469789271397
Epoch 4, Loss: 0.10660284879670072
Epoch 5, Loss: 0.08746009690710915
Epoch 6, Loss: 0.07567427212371429
Epoch 7, Loss: 0.06496873360650024
Epoch 8, Loss: 0.05941510789907411
Epoch 9, Loss: 0.05250461398492932
Epoch 10, Loss: 0.044819096597299185
Training Accuracy: 98.76833333333333 %
Test Accuracy: 97.52 %
Validation Accuracy: 97.52 %


In [5]:
import numpy as np
import os
import cv2

# Load the training data from the folder
train_folder = '/home/vaishnavi-moutam/Downloads/final/Training1'
X_train = []
y_train = []
for file in os.listdir(train_folder):
    img = cv2.imread(os.path.join(train_folder, file))
    if img is not None:  # Check if image is not empty
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_train.append(img)  # Append the flattened image to the list
        y_train.append(1)  # or 0, depending on the class label
    else:
        print(f"Error reading file {file}")  # Print an error message if the image is empty

# Convert the lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Normalize the pixel values to be between 0 and 1
X_train = X_train / 255.0

# Initialize the weights and bias for L2 regularization
weights_l2 = np.random.rand(X_train.shape[1])
bias_l2 = 0

# Initialize the weights and bias for L1 regularization
weights_l1 = np.random.rand(X_train.shape[1])
bias_l1 = 0

# Initialize the weights and bias for Dropout regularization
weights_dropout = np.random.rand(X_train.shape[1])
bias_dropout = 0

# Set the learning rate, regularization parameters, and number of iterations
learning_rate = 0.01
reg_param_l2 = 0.1  # L2 regularization parameter
reg_param_l1 = 0.01  # L1 regularization parameter
dropout_rate = 0.2  # Dropout rate
n_iters = 3

# Training loop with L2 regularization
for iter in range(n_iters):
    for i, x_i in enumerate(X_train):
        # Calculate the predicted output with L2 regularization
        predicted_l2 = np.where(np.dot(x_i, weights_l2) + bias_l2 >= 0, 1, -1)
        
        # Update the weights and bias with L2 regularization
        update_l2 = learning_rate * (y_train[i] - predicted_l2)
        weights_update_l2 = update_l2 * x_i - reg_param_l2 * weights_l2  # L2 regularization update
        weights_l2 += weights_update_l2
        bias_l2 += update_l2

    # Calculate the accuracy and loss with L2 regularization
    predictions_l2 = np.where(np.dot(X_train, weights_l2) + bias_l2 >= 0, 1, -1)
    accuracy_l2 = np.mean(predictions_l2 == y_train)
    loss_l2 = np.mean((predictions_l2 - y_train) ** 2) + 0.5 * reg_param_l2 * np.sum(weights_l2 ** 2)
    
    # Print the accuracy and loss with L2 regularization
    print(f'Iteration {iter+1}, L2 Regularization - Accuracy: {accuracy_l2:.2f}, Loss: {loss_l2:.2f}')

# Save the trained model with L2 regularization
np.save('weights_l2.npy', weights_l2)
np.save('bias_l2.npy', bias_l2)

# Training loop with L1 regularization
for iter in range(n_iters):
    for i, x_i in enumerate(X_train):
        # Calculate the predicted output with L1 regularization
        predicted_l1 = np.where(np.dot(x_i, weights_l1) + bias_l1 >= 0, 1, -1)
        
        # Update the weights and bias with L1 regularization
        update_l1 = learning_rate * (y_train[i] - predicted_l1)
        weights_update_l1 = update_l1 * x_i - reg_param_l1 * np.sign(weights_l1)  # L1 regularization update
        weights_l1 += weights_update_l1
        bias_l1 += update_l1

    # Calculate the accuracy and loss with L1 regularization
    predictions_l1 = np.where(np.dot(X_train, weights_l1) + bias_l1 >= 0, 1, -1)
    accuracy_l1 = np.mean(predictions_l1 == y_train)
    loss_l1 = np.mean((predictions_l1 - y_train) ** 2) + reg_param_l1 * np.sum(np.abs(weights_l1))
    
    # Print the accuracy and loss with L1 regularization
    print(f'Iteration {iter+1}, L1 Regularization - Accuracy: {accuracy_l1:.2f}, Loss: {loss_l1:.2f}')

# Save the trained model with L1 regularization
np.save('weights_l1.npy', weights_l1)
np.save('bias_l1.npy', bias_l1)

# Training loop with Dropout regularization
for iter in range(n_iters):
    for i, x_i in enumerate(X_train):
        # Apply dropout during training
        mask = np.random.binomial(1, 1 - dropout_rate, size=weights_dropout.shape)
        x_i_dropout = x_i * mask / (1 - dropout_rate)
        
        # Calculate the predicted output with Dropout regularization
        predicted_dropout = np.where(np.dot(x_i_dropout, weights_dropout) + bias_dropout >= 0, 1, -1)
        
        # Update the weights and bias without regularization (dropout already applied)
        update_dropout = learning_rate * (y_train[i] - predicted_dropout)
        weights_update_dropout = update_dropout * x_i_dropout
        weights_dropout += weights_update_dropout
        bias_dropout += update_dropout

    # Calculate the accuracy and loss with Dropout regularization
    predictions_dropout = np.where(np.dot(X_train, weights_dropout) + bias_dropout >= 0, 1, -1)
    accuracy_dropout = np.mean(predictions_dropout == y_train)
    loss_dropout = np.mean((predictions_dropout - y_train) ** 2)
    
    # Print the accuracy and loss with Dropout regularization
    print(f'Iteration {iter+1}, Dropout Regularization - Accuracy: {accuracy_dropout:.2f}, Loss: {loss_dropout:.2f}')

# Save the trained model with Dropout regularization
np.save('weights_dropout.npy', weights_dropout)
np.save('bias_dropout.npy', bias_dropout)

# Load the validation data from the folder
val_folder = '/home/vaishnavi-moutam/Downloads/final/Validation1'
X_val = []
y_val = []
for file in os.listdir(val_folder):
    img = cv2.imread(os.path.join(val_folder, file))
    if img is not None:  # Check if image is not empty
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_val.append(img)  # Append the flattened image to the list
        y_val.append(1)  # or 0, depending on the class label
    else:
        print(f"Error reading file {file}")  # Print an error message if the image is empty

# Convert the lists to numpy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)

# Normalize the pixel values to be between 0 and 1
X_val = X_val / 255.0

# Load the trained models with L1, L2, and Dropout regularization
weights_l2 = np.load('weights_l2.npy')
bias_l2 = np.load('bias_l2.npy')

weights_l1 = np.load('weights_l1.npy')
bias_l1 = np.load('bias_l1.npy')

weights_dropout = np.load('weights_dropout.npy')
bias_dropout = np.load('bias_dropout.npy')

# Evaluate the validation set accuracy for each regularization technique
predictions_l2 = np.where(np.dot(X_val, weights_l2) + bias_l2 >= 0, 1, -1)
accuracy_val_l2 = np.mean(predictions_l2 == y_val)
print(f'Validation Accuracy with L2 Regularization: {accuracy_val_l2:.2f}')

predictions_l1 = np.where(np.dot(X_val, weights_l1) + bias_l1 >= 0, 1, -1)
accuracy_val_l1 = np.mean(predictions_l1 == y_val)
print(f'Validation Accuracy with L1 Regularization: {accuracy_val_l1:.2f}')

predictions_dropout = np.where(np.dot(X_val, weights_dropout) + bias_dropout >= 0, 1, -1)
accuracy_val_dropout = np.mean(predictions_dropout == y_val)
print(f'Validation Accuracy with Dropout Regularization: {accuracy_val_dropout:.2f}')

# Load the testing data from the folder
test_folder = '/home/vaishnavi-moutam/Downloads/final/Testing1'
X_test = []
y_test = []
for file in os.listdir(test_folder):
    img = cv2.imread(os.path.join(test_folder, file))
    if img is not None:  # Check if image is not empty
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_test.append(img)  # Append the flattened image to the list
        y_test.append(1)  # or 0, depending on the class label
    else:
        print(f"Error reading file {file}")  # Print an error message if the image is empty

# Convert the lists to numpy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Normalize the pixel values to be between 0 and 1
X_test = X_test / 255.0

# Evaluate the test set accuracy for each regularization technique
predictions_l2_test = np.where(np.dot(X_test, weights_l2) + bias_l2 >= 0, 1, -1)
accuracy_test_l2 = np.mean(predictions_l2_test == y_test)
print(f'Test Accuracy with L2 Regularization: {accuracy_test_l2:.2f}')

predictions_l1_test = np.where(np.dot(X_test, weights_l1) + bias_l1 >= 0, 1, -1)
accuracy_test_l1 = np.mean(predictions_l1_test == y_test)
print(f'Test Accuracy with L1 Regularization: {accuracy_test_l1:.2f}')

predictions_dropout_test = np.where(np.dot(X_test, weights_dropout) + bias_dropout >= 0, 1, -1)
accuracy_test_dropout = np.mean(predictions_dropout_test == y_test)
print(f'Test Accuracy with Dropout Regularization: {accuracy_test_dropout:.2f}')


Error reading file MNIST
Iteration 1, L2 Regularization - Accuracy: 1.00, Loss: 0.00
Iteration 2, L2 Regularization - Accuracy: 1.00, Loss: 0.00
Iteration 3, L2 Regularization - Accuracy: 1.00, Loss: 0.00
Iteration 1, L1 Regularization - Accuracy: 1.00, Loss: 0.16
Iteration 2, L1 Regularization - Accuracy: 1.00, Loss: 0.15
Iteration 3, L1 Regularization - Accuracy: 1.00, Loss: 0.16
Iteration 1, Dropout Regularization - Accuracy: 1.00, Loss: 0.00
Iteration 2, Dropout Regularization - Accuracy: 1.00, Loss: 0.00
Iteration 3, Dropout Regularization - Accuracy: 1.00, Loss: 0.00
Error reading file MNIST
Validation Accuracy with L2 Regularization: 1.00
Validation Accuracy with L1 Regularization: 1.00
Validation Accuracy with Dropout Regularization: 1.00
Error reading file evaluation_results_l2.txt
Error reading file MNIST
Error reading file final_model_l2.h5
Error reading file evaluation_results_dropout.txt
Error reading file final_model_dropout.h5
Error reading file final_model_l1.h5
Error r

In [6]:
#Compare performance with various optimization algorithms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.datasets import ImageFolder
from PIL import Image
import os

# Define transformations
transform = Compose([
    Resize((28, 28)),
    ToTensor()
])

# Define data paths
training_path = "/home/vaishnavi-moutam/Downloads/final/Training1"
validation_path = "/home/vaishnavi-moutam/Downloads/final/Validation1"
testing_path = "/home/vaishnavi-moutam/Downloads/final/Testing1"

# Load training data
train_samples = []
for root, _, fnames in sorted(os.walk(training_path)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            train_samples.append((transform(img), 0))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")
train_loader = DataLoader(train_samples, batch_size=32, shuffle=True)

# Load validation data
val_samples = []
for root, _, fnames in sorted(os.walk(validation_path)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            val_samples.append((transform(img), 0))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")
val_loader = DataLoader(val_samples, batch_size=32, shuffle=False)

# Load testing data
test_samples = []
for root, _, fnames in sorted(os.walk(testing_path)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            test_samples.append((transform(img), 0))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")
test_loader = DataLoader(test_samples, batch_size=32, shuffle=False)

# Define model architecture
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

# Define optimizers to compare
optimizers = {
    'Adam': optim.Adam(model.parameters(), lr=0.001),
    'SGD': optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
    'RMSprop': optim.RMSprop(model.parameters(), lr=0.001)
}

# Training and evaluation loop for each optimizer
num_epochs = 3
for optimizer_name, optimizer in optimizers.items():
    print(f"Training with {optimizer_name} optimizer:")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)

        # Calculate training accuracy
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in train_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        train_acc = correct / total

        # Calculate validation accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_acc = correct / total

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # Evaluate the model on the test set
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_acc = correct / total
    print(f"Testing Accuracy with {optimizer_name} optimizer: {test_acc:.4f}")
    print()


Skipping /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-images-idx3-ubyte: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-images-idx3-ubyte'
Skipping /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-images-idx3-ubyte.gz: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-images-idx3-ubyte.gz'
Skipping /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-labels-idx1-ubyte: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-labels-idx1-ubyte'
Skipping /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-labels-idx1-ubyte.gz: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/t10k-labels-idx1-ubyte.gz'
Skipping /home/vaishnavi-moutam/Downloads/final/Training1/MNIST/raw/train-images-idx3-ubyte: cannot identify image file '/home/vaishnavi-moutam/Downloads/final/Training1/MNIST/